# Primary Value and Learned Value Learning Algorithm

In [ ]:
import dotdot
import leabra

### Layers

In [ ]:
# layer whose activity is set directly
timing   = leabra.Layer(6)
stimulus = leabra.Layer(1)
PVe      = leabra.Layer(1)

# layer who compute their own activities
lspec = leabra.LayerSpec(k=1)
PVi = leabra.layer(1, spec=lspec)
LVe = leabra.Layer(1, spec=lspec)
LVi = leabra.Layer(1, spec=lspec)
DA  = leabra.Layer(1, spec=lspec)

### Connections

In [ ]:
# stimulus -> PVi, LVe, LVi
delta_spec = leabra.ConnectionSpec(lrule='delta', proj='full')
conn_stim_PVi = leabra.Connection(stimulus, PVi, spec=delta_spec)
conn_stim_LVe = leabra.Connection(stimulus, LVe, spec=delta_spec)
conn_stim_LVi = leabra.Connection(stimulus, LVi, spec=delta_spec)

# timing -> PVi, LVe, LVi
conn_time_PVi = leabra.Connection(timing,   PVi, spec=delta_spec)
conn_time_LVe = leabra.Connection(timing,   LVe, spec=delta_spec)
conn_time_LVi = leabra.Connection(timing,   LVi, spec=delta_spec)

# magic connections:
# * PVe -> PVi
# * PVi -> LVe
# * PVi -> LVi
# * PVi, LVe, LVi -> DA


### Network

In [ ]:
network = leabra.Network(layers=[PVi, LVe, LVi], 
                         connections=[conn_stim_PVi, conn_stim_LVe, conn_stim_LVi,
                                      conn_time_PVi, conn_time_LVe, conn_time_LVi])

### Inputs

In [ ]:
def trial():
    # inputs for 5 settling periods (15 cycles each)
    stim_in = [[0.0], [1.0], [1.0], [1.0], [0.0]]  # stimulus inputs
    rewd_in = [[0.0], [0.0], [0.0], [1.0], [0.0]]  # reward inputs
    for t in range(5):
        timing.set_activities([0.0 for i in range(5)])
        timing.units[t].act = 1.0  # timing inputs
        stimulus.set_activities(stim_in[t])
        PVe.set_activities(rewd_in[t])